<a href="https://colab.research.google.com/github/MuriloWollmann/Projeto-Final-de-Fabrica-de-Software---Automatizacao-da-Busca-e-Tratamento-de-Dados-de-Saude-Publica/blob/main/PFS_Dados_de_Sa%C3%BAde_P%C3%BAblica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# FUNCIONANDO - BAIXA TODAS AS TABELAS POR ATENDIMENTO

%pip install -q google-colab-selenium
import google_colab_selenium as gs

import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

class WebScraper:
    def __init__(self):
        self.driver = gs.Chrome()
        self.wait = WebDriverWait(self.driver, 10)
        self.driver.get('https://datasus.saude.gov.br/informacoes-de-saude-tabnet/')

    def get_elemento(self, xpath):
        return self.wait.until(EC.presence_of_element_located((By.XPATH, xpath)))

    def clicar_elemento(self, xpath):
        elemento = self.get_elemento(xpath)
        elemento.click()
        return elemento

    def escrever_no_campo(self, xpath, texto):
        campo = self.get_elemento(xpath)
        campo.send_keys(texto)
        return campo

    def limpar_campo(self, xpath):
        campo = self.get_elemento(xpath)
        campo.clear()

    def aguardar_visibilidade(self, xpath):
        return self.wait.until(EC.visibility_of_element_located((By.XPATH, xpath)))

    def tirar_screenshot(self, nome_arquivo):
        self.driver.save_screenshot(nome_arquivo)

    def selecionar_multiplas_opcoes(self, *xpaths):
        action = ActionChains(self.driver)
        action.key_down(Keys.CONTROL)
        for xpath in xpaths:
            self.aguardar_visibilidade(xpath)
            elemento = self.get_elemento(xpath)
            action.click(elemento)
        action.key_up(Keys.CONTROL)
        action.perform()

    def trocar_para_nova_janela(self, janela_indice):
        nova_janela = self.driver.window_handles[janela_indice]
        self.driver.switch_to.window(nova_janela)

def main():
    try:
        # Inicializa o scraper
        scraper = WebScraper()

        # Começa no site do Tabnet

        # Clica em Epidemiológicas e Morbidade
        scraper.clicar_elemento('//*[@id="elementor-tab-title-2153"]')
        scraper.aguardar_visibilidade('//*[@id="elementor-tab-content-2153"]/ul/li[12]/span/a/span')

        # Clica Sistema de Informação do Câncer - SISCAN (colo do útero e mama)
        scraper.clicar_elemento('//*[@id="elementor-tab-content-2153"]/ul/li[12]/span/a/span')
        scraper.aguardar_visibilidade('//*[@id="infesq"]/input[8]')

        # Clica em Mamografia - Por local de atendimento
        scraper.clicar_elemento('//*[@id="infesq"]/input[8]')
        scraper.aguardar_visibilidade('//*[@id="mySelect"]/option[23]')

        # Clica em Rio Grande do Sul
        scraper.clicar_elemento('//*[@id="mySelect"]/option[23]')
        scraper.aguardar_visibilidade('//*[@id="L"]/option[7]')

        # Escolhe a opção de linha: Faixa Etária
        scraper.clicar_elemento('//*[@id="L"]/option[7]')

        # Escolhe a opção de coluna: Ano de Competência
        scraper.clicar_elemento('//*[@id="C"]/option[2]')

        # Escolhe os anos: 2023, 2022, 2021, 2020, 2019
        scraper.clicar_elemento('//*[@id="A"]/option[1]')
        scraper.selecionar_multiplas_opcoes('//*[@id="A"]/option[2]', '//*[@id="A"]/option[3]', '//*[@id="A"]/option[4]', '//*[@id="A"]/option[5]', '//*[@id="A"]/option[6]')
        scraper.aguardar_visibilidade('//*[@id="fig2"]')

        # ------------ MUNICÍPIOS ------------
        # Cruz Alta: //*[@id="S2"]/option[127]
        # Giruá:     //*[@id="S2"]/option[183]
        # Ijuí:      //*[@id="S2"]/option[204]
        # Panambi:   //*[@id="S2"]/option[299]

        # Clica no filtro de município e escreve "Cruz Alta"
        scraper.clicar_elemento('//*[@id="fig2"]')
        scraper.clicar_elemento('//*[@id="S2"]/option[1]')
        campo_entrada = scraper.escrever_no_campo('//*[@id="pesqmes2"]', "Cruz Alta")
        scraper.aguardar_visibilidade('//*[@id="S2"]/option[127]')
        scraper.clicar_elemento('//*[@id="S2"]/option[127]')

        # Escolhe a opção: feminino
        scraper.clicar_elemento('//*[@id="fig8"]')
        scraper.clicar_elemento('//*[@id="S8"]/option[1]')
        scraper.clicar_elemento('//*[@id="S8"]/option[3]')

        # Clicar no botão de gerar tabela
        scraper.clicar_elemento('/html/body/center[2]/div/form/div[3]/div[2]/div[3]/input[1]')

        # Clica no filtro de município de prestação de serviço e escreve "Giruá"
        scraper.clicar_elemento('//*[@id="S2"]/option[127]')
        scraper.limpar_campo('//*[@id="pesqmes2"]')
        campo_entrada = scraper.escrever_no_campo('//*[@id="pesqmes2"]', "Giruá")
        scraper.clicar_elemento('//*[@id="S2"]/option[183]')

        # Clicar no botão de gerar tabela
        scraper.clicar_elemento('/html/body/center[2]/div/form/div[3]/div[2]/div[3]/input[1]')

        # Clica no filtro de município de prestação de serviço e escreve "Ijuí"
        scraper.clicar_elemento('//*[@id="S2"]/option[183]')
        scraper.limpar_campo('//*[@id="pesqmes2"]')
        campo_entrada = scraper.escrever_no_campo('//*[@id="pesqmes2"]', "Ijuí")
        scraper.clicar_elemento('//*[@id="S2"]/option[204]')

        # Clicar no botão de gerar tabela
        scraper.clicar_elemento('/html/body/center[2]/div/form/div[3]/div[2]/div[3]/input[1]')

        # Clica no filtro de município de prestação de serviço e escreve "Panambi"
        scraper.clicar_elemento('//*[@id="S2"]/option[204]')
        scraper.limpar_campo('//*[@id="pesqmes2"]')
        campo_entrada = scraper.escrever_no_campo('//*[@id="pesqmes2"]', "Panambi")
        scraper.clicar_elemento('//*[@id="S2"]/option[299]')

        # Clicar no botão de gerar tabela
        scraper.clicar_elemento('/html/body/center[2]/div/form/div[3]/div[2]/div[3]/input[1]')

        # Trocar para a nova janela/aba (Panambi)
        scraper.trocar_para_nova_janela(4)
        scraper.aguardar_visibilidade('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        # Baixar csv.
        scraper.clicar_elemento('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        scraper.tirar_screenshot('/content/captura_de_tela1.png')

        # Trocar para a nova janela/aba (Cruz Alta)
        scraper.trocar_para_nova_janela(3)
        scraper.aguardar_visibilidade('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        # Baixar csv.
        scraper.clicar_elemento('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        scraper.tirar_screenshot('/content/captura_de_tela2.png')

        # Trocar para a nova janela/aba (Giruá)
        scraper.trocar_para_nova_janela(2)
        scraper.aguardar_visibilidade('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        # Baixar csv.
        scraper.clicar_elemento('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        scraper.tirar_screenshot('/content/captura_de_tela3.png')

        # Trocar para a nova janela/aba (Ijuí)
        scraper.trocar_para_nova_janela(1)
        scraper.aguardar_visibilidade('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        # Baixar csv.
        scraper.clicar_elemento('/html/body/center[2]/div[2]/table/tbody/tr/td[2]/a')

        scraper.tirar_screenshot('/content/captura_de_tela4.png')

    except Exception as e:
        print(f"Erro no processo principal: {e}")

if __name__ == "__main__":
    main()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.7/475.7 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.4 MB/s eta 0:00:00


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>